## Atividades Bases da Computação III

Curso: Sistemas de Informação (3º Semestre)

------------

### Atividade 7

Essa atividade visa estudarmos um pouco mais os conteúdos sobre árvores vistos em aula e no material.
Novamente, a ideia é implementar uma estrutura de dados em árvore em Python. 

Peço que implementem um novo notebook no google colab com as seguintes classes: 
- Nó (com os atributos chave, altura, fator de balanceamento, pai, filho 
esquerdo e filho direito), e 
- Árvore (com o atributo raiz (que tem o tipo nó)) 

A árvore deve ter os seguintes métodos: 
- Inserir nó em uma árvore AVL 
- Remover um nó em uma árvore AVL 

Repare que para programar os métodos de inserção e remoção em árvore AVL, são necessários outros métodos adicionais que mantenham a árvore AVL balanceada e que também devem ser incorporados à classe árvore. 

Faça testes, comente o código, use os textos entre os códigos para explicar o que tem em cada chunk. 

### Definição das Classes

In [4]:
class No:
    def __init__(self, chave):
        self.chave = chave
        self.altura = 1
        self.fator_balanceamento = 0 
        self.pai = None
        self.esquerdo = None
        self.direito = None

In [5]:
class ArvoreAVL:
    def __init__(self):
        self.raiz = None

    # ---------- Métodos Auxiliares ---------- #
    
    def calcular_altura(self, no):
        if no is None:
            return 0
        return no.altura

    def atualizar_altura(self, no):
        if no:
            no.altura = 1 + max(self.calcular_altura(no.esquerdo), self.calcular_altura(no.direito))

    def calcular_fator_balanceamento(self, no):
        if no is None:
            return 0
        return self.calcular_altura(no.esquerdo) - self.calcular_altura(no.direito)

    def rotacionar_direita(self, y):
        x = y.esquerdo
        T2 = x.direito

        # Realizando a rotação
        x.direito = y
        y.esquerdo = T2

        # Atualizando pais
        if T2:
            T2.pai = y
        x.pai = y.pai
        y.pai = x

        # Atualizando alturas
        self.atualizar_altura(y)
        self.atualizar_altura(x)

        return x

    def rotacionar_esquerda(self, x):
        y = x.direito
        T2 = y.esquerdo

        # Realizando a rotação
        y.esquerdo = x
        x.direito = T2

        # Atualizando pais
        if T2:
            T2.pai = x
        y.pai = x.pai
        x.pai = y

        # Atualizando alturas
        self.atualizar_altura(x)
        self.atualizar_altura(y)

        return y

    def balancear(self, no):
        # Atualiza o fator de balanceamento e verifica se precisa de rotação
        self.atualizar_altura(no)
        fator_balanceamento = self.calcular_fator_balanceamento(no)

        # Se a árvore ficou desbalanceada, faz-se a rotação necessária
        if fator_balanceamento > 1:
            if self.calcular_fator_balanceamento(no.esquerdo) < 0:
                # Rotação Esquerda-Direita
                no.esquerdo = self.rotacionar_esquerda(no.esquerdo)
            # Rotação Direita
            return self.rotacionar_direita(no)

        if fator_balanceamento < -1:
            if self.calcular_fator_balanceamento(no.direito) > 0:
                # Rotação Direita-Esquerda
                no.direito = self.rotacionar_direita(no.direito)
            # Rotação Esquerda
            return self.rotacionar_esquerda(no)

        return no

    # ---------- Métodos de Inserção e Remoção ---------- #

    def inserir(self, chave):
        # Método principal para inserção de nó na árvore
        if not self.raiz:
            self.raiz = No(chave)
        else:
            self.raiz = self._inserir_recursivo(self.raiz, chave)

    def _inserir_recursivo(self, no, chave):
        if no is None:
            return No(chave)

        if chave < no.chave:
            no.esquerdo = self._inserir_recursivo(no.esquerdo, chave)
            no.esquerdo.pai = no
        else:
            no.direito = self._inserir_recursivo(no.direito, chave)
            no.direito.pai = no

        # Balanceando a árvore após a inserção
        return self.balancear(no)

    def remover(self, chave):
        # Método principal para remoção de nó da árvore
        if self.raiz:
            self.raiz = self._remover_recursivo(self.raiz, chave)

    def _remover_recursivo(self, no, chave):
        if no is None:
            return no

        # Encontrar o nó a ser removido
        if chave < no.chave:
            no.esquerdo = self._remover_recursivo(no.esquerdo, chave)
        elif chave > no.chave:
            no.direito = self._remover_recursivo(no.direito, chave)
        else:
            # Nó encontrado, caso 1: Nó folha (sem filhos)
            if no.esquerdo is None and no.direito is None:
                return None
            # Caso 2: Nó com um filho
            elif no.esquerdo is None:
                return no.direito
            elif no.direito is None:
                return no.esquerdo
            # Caso 3: Nó com dois filhos
            min_node = self._minimo(no.direito)
            no.chave = min_node.chave
            no.direito = self._remover_recursivo(no.direito, min_node.chave)

        # Balanceando a árvore após a remoção
        return self.balancear(no)

    def _minimo(self, no):
        while no.esquerdo:
            no = no.esquerdo
        return no

    # ---------- Métodos para Imprimir e Verificar ---------- #
    def imprimir_em_ordem(self):
        elementos = []
        self._em_ordem(self.raiz, elementos)
        return elementos

    def _em_ordem(self, no, elementos):
        if no:
            self._em_ordem(no.esquerdo, elementos)
            elementos.append(no.chave)
            self._em_ordem(no.direito, elementos)

    def verificar_balanceamento(self):
        # Verifica se a árvore está balanceada
        if not self.raiz:
            return True
        return self._verificar_balanceamento(self.raiz)

    def _verificar_balanceamento(self, no):
        if no is None:
            return True
        fator_balanceamento = self.calcular_fator_balanceamento(no)
        return (abs(fator_balanceamento) <= 1 and
                self._verificar_balanceamento(no.esquerdo) and
                self._verificar_balanceamento(no.direito))

### Testagem

In [6]:
avl = ArvoreAVL()

avl.inserir(10)
avl.inserir(20)
avl.inserir(5)
avl.inserir(6)
avl.inserir(15)

print("Elementos da Árvore AVL em ordem:", avl.imprimir_em_ordem())

avl.remover(10)
print("Elementos após remoção do nó 10:", avl.imprimir_em_ordem())

print("A árvore está balanceada?", avl.verificar_balanceamento())


Elementos da Árvore AVL em ordem: [5, 6, 10, 15, 20]
Elementos após remoção do nó 10: [5, 6, 15, 20]
A árvore está balanceada? True
